### 3 GL Summary
<br>


In [1]:
import pandas as pd
import numpy as np
import os
pd.options.display.float_format = '{:,.2f}'.format

####  Import GL

In [2]:
os.chdir('/Users/lorenkarish/Desktop/JBS/Bank')

gl = pd.read_excel('gl_Aug-Dec.xlsx', header=0) #<-----updated to include July

gl.rename({'period': 'month'}, axis=1, inplace=True)
gl['month'] = gl['month'].str[5:]
gl.drop(['ref_num', 'seq_num','detail_id','type'], axis=1,inplace=True)

calendar = {'07': 'Jul','08': 'Aug', '09': 'Sep', '10': 'Oct', '11': 'Nov', '12': 'Dec'}

category = {'APC': 'AP Check', 'AR-CHECK': 'cash-in', 'AR-OTHER': 'cash-in', 
       'JE': 'JE', 'JEREV': 'reversal','PRC':'payroll', 'EXPCK':'pay'}

gl['category'] = gl['source'].map(category)
gl['month'] = gl['month'].map(calendar)

<br>
<br>

#### get July check data 


In [3]:
df = gl[(gl['month'] == 'Jul') & (gl['source'] == 'APC')].copy()
df['check'] = df['desc'].str.extract('(\d+)')
Jul_AP = df[df['check'].str.len() > 3]
%store Jul_AP

gl.drop(gl[gl['month'] == 'Jul'].index, inplace=True)
gl = gl.reset_index(drop=True)

Stored 'Jul_AP' (DataFrame)


<br>

#### Balances

In [4]:
begin_bal = {'Aug' : -18676.32, 'Sep' : -395433.52, 'Oct' : -67183.45,  
            'Nov': -448413.32, 'Dec': -1040039.45} 

end_bal = {'Aug' : -395433.52, 'Sep' : -67183.45, 'Oct': -448413.32,  
            'Nov': -1040039.45, 'Dec': -836464.73}

b_bal = pd.Series(begin_bal)
e_bal = pd.Series(end_bal)    

<br>

### Loan
#### Aug

In [5]:
LOC_08adv = gl[((gl['desc'].str.startswith('REC RLOC TRANSACTIONS')) & (gl['amount'] > 0)) | \
                (gl['desc'].str.contains('Xfer oper / Loc'))] \
                .groupby('month')['amount'].agg('sum')

Aug = gl[(gl['category'] == 'JE') & (gl['month'] == 'Aug') & (gl['amount'] > 0) 
          & ~(gl['desc'].str.contains('INTEREST'))]

LOC_AugMiscAdv = Aug[(Aug['desc'].str.contains('LOC')) & 
                     ~(Aug['desc'].str.contains('|'.join(['TRANSACTIONS', 'ATTF']))) |
                      (Aug.index == 119 )] \
                      .groupby('month')['amount'].agg('sum')
#<<-------------->>

LOC_08pay = gl[(gl['desc'].str.startswith('REC RLOC TRANSACTIONS')) & (gl['amount'] < 0)] \
               .groupby('month')['amount'].agg('sum')


Aug = gl[(gl['category'] == 'JE') & (gl['month'] == 'Aug') & (gl['amount'] < 0) 
          & ~(gl['desc'].str.contains('INTEREST'))]

LOC_AugMiscPay = Aug[(Aug['desc'].str.contains('LOC')) & ~(Aug['desc'].str.contains('TRANSACTIONS'))] \
                      .groupby('month')['amount'].agg('sum')
#<<-------------->>

LOC_08pay = + LOC_08pay + LOC_AugMiscPay
LOC_08adv = + LOC_08adv + LOC_AugMiscAdv

#### Sep

In [6]:
LOC_09adv = gl[(gl['desc'].str.startswith('LOC TRANSFER')) & (gl['amount'] > 0)] \
              .groupby('month')['amount'].agg('sum')

LOC_09pay = gl[(gl['desc'].str.startswith('LOC TRANSFER')) & (gl['amount'] < 0)] \
               .groupby('month')['amount'].agg('sum')

Sep_Misc = gl[(gl['category'] == 'JE') & (gl['month'] == 'Sep') & (gl['desc'].str.contains('LOC')) \
          & ~(gl['desc'].str.contains('TRANSFER'))] #LOC Annual Renewal: -$11373.32 + dr&cr for same

#### Oct

In [7]:
JE = gl[gl['category'] == 'JE']

LOC_10adv = JE[(JE['month'] == 'Oct') & (JE['desc'] == 'LOC CREDITS')] \
               .groupby('month')['amount'].agg('sum')

LOC_10pay = JE[(JE['month'] == 'Oct') & (JE['desc'] == 'LOC REPAYMENT')] \
               .groupby('month')['amount'].agg('sum')

#### Nov

In [8]:
LOC_11adv = JE[(JE['month'] == 'Nov') & (JE['desc'] == 'ADV FROM LOC')] \
               .groupby('month')['amount'].agg('sum')

LOC_11pay = JE[(JE['month'] == 'Nov') & (JE['desc'] == 'PYMT TO LOC')] \
               .groupby('month')['amount'].agg('sum')

#### Dec

In [9]:
LOC_12adv = JE[((JE['month'] == 'Dec') & (JE['desc'].str.contains(r'^(?=.*ADVANCE)(?=.*LOC)'))) | \
                (JE.index == 1017)] \
                .groupby('month')['amount'].agg('sum') 

LOC_12pay = JE[(JE['month'] == 'Dec') & (JE['desc'] == 'PAYMENT TO LOC')] \
               .groupby('month')['amount'].agg('sum')

<br>

In [10]:
LOC_adv = pd.concat([LOC_08adv, LOC_09adv, LOC_10adv, LOC_11adv, LOC_12adv])
LOC_pay = pd.concat([LOC_08pay, LOC_09pay, LOC_10pay, LOC_11pay, LOC_12pay])

### Checks

distinction b/w 3 and 5-digit check numbers 

In [11]:
checks_A = gl[(gl['desc'].str.contains(r'\d{5}')) & (gl['category'] == 'AP Check') & \
              ~(gl['desc'].str.contains('void'))] 

checks_B = gl[(gl['category'] == 'AP Check') & ~(gl.index.isin(checks_A.index)) & \
             ~(gl['desc'].str.contains('void'))]

checks_Op = checks_A.groupby('month')['amount'].agg('sum')
checks_Other = checks_B.groupby('month')['amount'].agg('sum')

### Voided Checks
some checks are voided w/ multiple entries (e.g. original amt of 100, reversals of 75 and 25)
 - 5-digit: 227 entries 
 - 3-digit: 75 entries 
 

In [12]:
voids = gl[(gl['source'] == 'APC') & (gl['desc'].str.contains('void'))]
voids_5 = voids[voids['desc'].str.contains(r'\d{5}')]

voids_3 = voids[~(voids.index.isin(voids_5.index))]

voids_5 = voids_5.groupby('month')['amount'].agg('sum')
voids_3 = voids_3.groupby('month')['amount'].agg('sum')

### Other_Payments
includes:
- 376K: Capital One (Aug, Sep)
- 165K: ABT payment

In [13]:
other_pay = gl[(gl['category'] == 'JE') & (gl['desc'].str.contains('Capital'))] \
                .groupby('month')['amount'].agg('sum')


### Deposits
 - AR-Check
 - AR-Other

In [14]:
AR_DepA = gl[gl['source'] == 'AR-CHECK'].groupby('month')['amount'].agg('sum')
AR_DepB = gl[gl['source'] == 'AR-OTHER'].groupby('month')['amount'].agg('sum')
cash_in = AR_DepA.add(AR_DepB, fill_value=0)


### Take_Inventory

- this needs to be executed twice (see code comment)

In [20]:
LOC_08in = gl[((gl['desc'].str.startswith('REC RLOC TRANSACTIONS')) & (gl['amount'] > 0)) | \
                (gl['desc'].str.contains('Xfer oper / Loc'))]

Aug = gl[(gl['category'] == 'JE') & (gl['month'] == 'Aug') & (gl['amount'] > 0) 
          & ~(gl['desc'].str.contains('INTEREST'))].copy()

LOC_08_Misc_In = Aug[(Aug['desc'].str.contains('LOC')) & 
                     ~(Aug['desc'].str.contains('|'.join(['TRANSACTIONS', 'ATTF']))) |
                      (Aug.index == 119 )]

LOC_08out = gl[(gl['desc'].str.startswith('REC RLOC TRANSACTIONS')) & (gl['amount'] < 0)]

Aug = gl[(gl['category'] == 'JE') & (gl['month'] == 'Aug') & (gl['amount'] < 0) 
          & ~(gl['desc'].str.contains('INTEREST'))].copy()

LOC_08_Misc_Out = Aug[(Aug['desc'].str.contains('LOC')) & ~(Aug['desc'].str.contains('TRANSACTIONS'))]

LOC_09in = gl[(gl['desc'].str.startswith('LOC TRANSFER')) & (gl['amount'] > 0)]
LOC_09out = gl[(gl['desc'].str.startswith('LOC TRANSFER')) & (gl['amount'] < 0)]


JE = gl[gl['category'] == 'JE'].copy()
LOC_10in = JE[(JE['month'] == 'Oct') & (JE['desc'] == 'LOC CREDITS')]
LOC_10out = JE[(JE['month'] == 'Oct') & (JE['desc'] == 'LOC REPAYMENT')] 


LOC_11in = JE[(JE['month'] == 'Nov') & (JE['desc'] == 'ADV FROM LOC')] 
LOC_11out = JE[(JE['month'] == 'Nov') & (JE['desc'] == 'PYMT TO LOC')]

LOC_12in = JE[((JE['month'] == 'Dec') & (JE['desc'].str.contains(r'^(?=.*ADVANCE)(?=.*LOC)'))) | \
                (JE.index == 1017)]

LOC_12out = JE[(JE['month'] == 'Dec') & (JE['desc'] == 'PAYMENT TO LOC')]



checks_1 = gl[(gl['desc'].str.contains(r'\d{5}')) & (gl['category'] == 'AP Check') & \
              ~(gl['desc'].str.contains('void'))] 

checks_2 = gl[(gl['category'] == 'AP Check') & ~(gl.index.isin(checks_1.index)) & \
             ~(gl['desc'].str.contains('void'))]


voids = gl[(gl['source'] == 'APC') & (gl['desc'].str.contains('void'))]
voids_five = voids[voids['desc'].str.contains(r'\d{5}')]
voids_three = voids[~(voids.index.isin(voids_5.index))]


noncheck_pay = gl[(gl['category'] == 'JE') & (gl['desc'].str.contains('Capital'))]


AR_Dep_A = gl[gl['source'] == 'AR-CHECK']
AR_Dep_B = gl[gl['source'] == 'AR-OTHER']

misc_deps = gl[(gl['category'] == 'JE') & (gl['desc'].str.contains('Iron Ox')) |
                ((gl['month'] == 'Dec') &(gl['desc'] == 'ERC PAYMENT')) | 
                ((gl['month'] == 'Dec') &(gl['desc'] == 'CORRECTING DEPOSIT')) |
                (gl['desc'].str.contains('COBRA'))]


#<---- PR_X has to be removed the 1st time this cell is executed
master = pd.concat([LOC_08in, LOC_08_Misc_Out, LOC_08out, LOC_08_Misc_In, LOC_09in, LOC_09out, \
                    LOC_10in, LOC_10out, LOC_11in, LOC_11out, LOC_12in, LOC_12out,  checks_1, \
                    checks_2, voids_five, voids_three, AR_Dep_A, AR_Dep_B, misc_deps,noncheck_pay,PR_X])

done = list(master.index)
to_do = gl[~(gl.index.isin(done))].copy()

# these are duplicate transactions that net to zero (contains orig entry + reversal)
dupe_JEs = [124,125,379,380,376,377,378,381,854,855,1020,1021,1022,1023,1024,1025,
           957,1038,185,186]

to_do = to_do[~(to_do.index.isin(dupe_JEs))].copy()

<br>

### PR Transfer

In [17]:
out_of_scope = [175,388,855,1073,957]
Aug_PR_X_401 = [18,104,117,173,12]


big_debits = to_do[to_do['amount'] < -70000].copy()
PR_X_a = big_debits[~(big_debits.index.isin(out_of_scope))]
PR_X_b = to_do[to_do.index.isin(Aug_PR_X_401)]
PR_X = pd.concat([PR_X_a,PR_X_b], axis=0)
PR_Xfer = PR_X.groupby('month')['amount'].agg('sum')

<br>

### Other Deposits
includes:
- 163K, Iron Ox (Aug, Sep)
- 242K, ERC payment; 438K w/ -196K Correction (both Dec)
- 8K: Cobra Payments

In [18]:
other_deps = gl[(gl['category'] == 'JE') & (gl['desc'].str.contains('Iron Ox')) |
                ((gl['month'] == 'Dec') &(gl['desc'] == 'ERC PAYMENT')) | 
                ((gl['month'] == 'Dec') &(gl['desc'] == 'CORRECTING DEPOSIT')) |
                (gl['desc'].str.contains('COBRA'))] \
                .groupby('month')['amount'].agg('sum')


more_deps = to_do[to_do['amount'] > 0].groupby('month')['amount'].agg('sum')

other_deps = other_deps.add(more_deps, fill_value=0)

### Other_Payments_part2

 - expands on criteria used above

In [19]:
other_pay_a = gl[(gl['category'] == 'JE') & (gl['desc'].str.contains('Capital'))] \
                .groupby('month')['amount'].agg('sum')

other_pay_b = to_do[to_do['amount'] < 0].groupby('month')['amount'].agg('sum')

other_pay = other_pay_a.add(other_pay_b, fill_value=0)

<br>



###  GL Summary

In [21]:
gl_summary = pd.concat([b_bal, cash_in, other_deps, checks_Op, voids_5, checks_Other, voids_3, other_pay, \
                        LOC_adv, LOC_pay, PR_Xfer, e_bal],axis=1)

gl_summary.columns = ['Beg','Deposits_AR','Other_Deposits','Checks_Op','Voids_Op','Checks_Other', \
                      'Voids_Other','Other_Payments','CL_Adv','CL_Pay','PR_x','End']

gl_summary = gl_summary.fillna(0)
gl_summary = gl_summary.T
gl_summary

,Aug,Sep,Oct,Nov,Dec
Beg,"-18,676.32","-395,433.52","-67,183.45","-448,413.32","-1,040,039.45"
Deposits_AR,"2,219,706.33","1,375,376.37","1,178,008.49","2,398,589.41","2,478,939.92"
Other_Deposits,"90,148.75","81,966.71","17,012.83","34,619.01","326,129.94"
Checks_Op,"-1,911,038.20","-160,823.82","-415,618.05","-1,271,919.96","-1,793,067.23"
Voids_Op,"385,114.14","11,674.95","114,854.69","59,657.63","1,029,621.96"
Checks_Other,"-975,969.23","-50,520.47","-970,118.55","-793,378.00","-1,697,038.51"
Voids_Other,"46,054.33","14,490.10","113,858.03","48,588.84","194,411.44"
Other_Payments,"-681,673.32","-293,219.73","-423,392.17","-589,965.33","-546,422.80"
CL_Adv,"1,645,500.00","1,269,000.00","884,500.00","899,500.00","1,020,500.00"
CL_Pay,"-1,194,600.00","-1,919,694.04","-880,335.14","-1,377,317.73","-809,500.00"


<br>


<br>

#### create Amount Frequency structure
 - added for use in Part 4

In [22]:
more_deps = to_do[to_do['amount'] > 0]

other_pay_a = gl[(gl['category'] == 'JE') & (gl['desc'].str.contains('Capital'))] 
other_pay_b = to_do[to_do['amount'] < 0]

other_pay_both = pd.concat([other_pay_a, other_pay_b])

In [23]:
credits = pd.concat([AR_Dep_A, AR_Dep_B, misc_deps, more_deps]) 

chex = pd.concat([checks_1, voids_five])

other_pay = pd.concat([checks_2, voids_three,other_pay_both]) 

loan_a = pd.concat([LOC_08in, LOC_08_Misc_In, LOC_09in, LOC_10in, LOC_11in, LOC_12in])
loan_p = pd.concat([LOC_08out, LOC_08_Misc_Out, LOC_09out, LOC_10out, LOC_11out, LOC_12out])

PR_Transfers = PR_X                      

                   
                   
gl_cats = {'Credits':  credits, 'Checks':  chex, 'Other_Pay':  other_pay,
       'CL_Adv':  loan_a, 'CL_Pay':  loan_p, 'PR_x': PR_Transfers}

gl_pool = {}

for k,v in gl_cats.items():
    mo_hist = []
    for month in list(gl['month'].unique()):
        results = v[v['month'] == month].copy() 
        mo_hist.append(results) 

    gl_pool[k] = mo_hist  

In [24]:
%store gl
%store gl_summary
%store gl_pool 

Stored 'gl' (DataFrame)
Stored 'gl_summary' (DataFrame)
Stored 'gl_pool' (dict)
